In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *


In [ ]:
spark=SparkSession.builder.appName('desafio-python').getOrCreate()

In [ ]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        white-space: pre !important;
    }
    pre { white-space: pre !important; }
</style>
""")

In [4]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 30)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 5)

In [5]:
dataframes_dict = {}
dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/bkpf_vbrk.parquet")
dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbpa_kna1.parquet")
dataframes_dict['materiales']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbrp_mara_marm.parquet")
dataframes_dict['paises']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/T005T.parquet")


In [6]:
dataframes_dict['metadata_de_facturas']
#1) Factura,Clase_factura

22/08/24 16:01:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Mandante,Sociedad,N_documento,Ejercicio,Clase_documento,Fecha_documento,Fecha_contab,Per_contable,Registrado_el,Modificado_el,Nombre_usuario,Referencia,Moneda,Moneda_grupo,Status_doc,Op_referencia,Clv_referencia,Entidad_CP,Moneda_local,Tipo_cotizacion1,Tipo_cotizacion2,Soc_desencaden,T_C_para_imptos,Motiv_anulacion,Num_de_tienda,Ctd_paginas,Ledger,Grupo_ledgers,FechDeclImpuest,Contab_dividida,Tipo_cotizacion,Tipo_de_orden,Motivo,Codigo_transaccion,Factura,Clase_factura,Organiz_ventas,Canal_distrib,Esquema_calculo,Condicion_doc,Condicion_expedicion,Fecha_factura,Grupo_precios,Grupo_clientes,Zona_de_ventas,Lista_precios,Incoterms,Incoterms_2,Status_contab,Camb_p_contab,Dias_val_adic,Fecha_de_valor,Cond_pago,Via_pago,Gpo_imputacion,Pais_receptor,Region,Cl_Fiscal_1_Clt,Valor_neto,Crit_agrupam,Creado_por,Hora,Creado_el,Pagador,Solicitante,Moneda_estad,N_com_exterior,DocFacturAnulad,Acuerdo,Cl_lista_fact,Fec_fact_lst_fc,Sector,Area_ctrl_cdto,Cta_credito,Cambio,JerTip_DetPrec,N_de_pedido,PSdeterm_impto,Origen_N_I_F,Pais_N_I_F,Asignacion,Impte_impuesto,Sistema_logico,Fe_conversion,TpCambCartCred,Refer_pago
400,127,0005047168,2020,KB,null,null,12,2020.12.01,2020.12.07,OPERACIONES,41279 1,CLP,null,null,CTE,41279 1 1,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047181,2020,HR,null,null,12,2020.12.01,1900.01.01,KTAPIA,FINI,CLP,null,null,BKPF,0005047181127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,FB01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047182,2020,HR,null,null,12,2020.12.01,1900.01.01,KTAPIA,FINI,CLP,null,null,BKPF,0005047182127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,FB01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047202,2020,HR,null,null,12,2020.12.01,1900.01.01,KTAPIA,FINI,CLP,null,null,BKPF,0005047202127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,FB01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047335,2020,KA,null,null,12,2020.12.01,1900.01.01,AREBOLLEDO,3970823468-4,CLP,null,null,BKPFF,0005047335127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,ZFI1165,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [7]:
df_metadata_facturas = dataframes_dict['metadata_de_facturas'].where(col('Mandante') == '400')

In [23]:
dataframes_dict['facturas_por_cliente']
# 1) Posicion_de_Factura,

Numero_de_Cliente,Mandante,Factura,Posicion_de_Factura,Tipo_de_Cliente,Direccion,Punto_descarga,Clave_pais,Indicador_direccion,Descr_cliente_socio,Zona_transporte,Numero_personal,Cliente_Solicitante,Cliente_Pagador,Cliente_Destinatario_de_Mercancia,Cliente_Destinatario_de_factura,Representante_de_ventas,Pais,Nombre,Poblacion,Codigo_postal,Region,Conc_busq,Calle,Telefono,Nombre_1,Poblacion_1,Tratamiento,Creado_el,Creado_por,Puestos_de_descarga,Grupo_de_ceuntas,Acreedor,Nombre_3,Nombre_4,Distrito,Clave_de_idioma,N_ident_fis_1,Telefono_2,Zona_de_transporte,Sociedad_GL_asociada,Atributo_1,Persona_fisica
164952,400,0150190076,0,AG,0001299162,null,CL,D,null,08102,0,164952,null,null,null,null,CL,Reciclaje de Metales Ingeni...,CORONEL,null,08,REMET,Av Forestal # 947 Parque I...,41-2751070,RECICLAJE DE METALES INGE,CORONEL,Sres.,2016.12.28,MDIAZB,null,CL01,null,null,null,null,S,76368838-0,null,08102,null,null,null
164952,400,0150190076,0,RG,0001299162,null,CL,D,null,08102,0,null,164952,null,null,null,CL,Reciclaje de Metales Ingeni...,CORONEL,null,08,REMET,Av Forestal # 947 Parque I...,41-2751070,RECICLAJE DE METALES INGE,CORONEL,Sres.,2016.12.28,MDIAZB,null,CL01,null,null,null,null,S,76368838-0,null,08102,null,null,null
164952,400,0150190076,0,RE,0001299162,null,CL,D,null,08102,0,null,null,null,164952,null,CL,Reciclaje de Metales Ingeni...,CORONEL,null,08,REMET,Av Forestal # 947 Parque I...,41-2751070,RECICLAJE DE METALES INGE,CORONEL,Sres.,2016.12.28,MDIAZB,null,CL01,null,null,null,null,S,76368838-0,null,08102,null,null,null
null,400,0150190076,0,VE,null,null,CL,D,null,null,99999999,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,400,0150190076,10,VD,null,null,CL,D,null,null,99999999,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [9]:
df_facturas_por_cliente = dataframes_dict['facturas_por_cliente'].where(col('Mandante') == '400')

In [10]:
dataframes_dict['materiales']
#1) Material, Volumen, Unidad_de_medida_Volumen_Original

Material,Mandante,Factura,Posicion,Posicion_sup,Ctd_facturada,UM_venta,UM_base,Ctd_necesaria,Division,Tp_cambio,Valor_neto,Causante,Posicion1,Doc_modelo,Posicion_modelo,Documento_venta,Posicion2,Denominacion,Material_precio,Lote,Grupo_articulos,Jquia_productos,Puesto_exped,Sector,Centro,Pais,Reg_ce_sum,Gr_materiales,Gr_imput_mat,Centro_coste,Grupo_rappels,Grupo_comision,Gr_vendedores,Oficina_ventas,Sector2,Cl_valoracion,Almacen,Costes_internos,CeBe,Gr_materiales_1,Asociacion_posicion_volumen,Gr_materiales_2,Gr_materiales_3,Plazo_entrega,Fecha_entrega,Creado_el,Creado_por,Ultima_modif,Modificado_por,Stat_act_comp,Stat_actual,Tipo_material,Ramo,N_Material_ant,UM_de_pedido,Numero_hojas,Fabr_insp,Tamano_Dimens,Denom_estandar,CvValoresCompr,Peso_bruto,Peso_neto,Unidad_de_peso,Cond_almacenaje,Nivel_plf_nec,Gpo_transporte,N_sust_peligr,Cant_vales_EM,Codigo_EAN_UPC,Tipo_EAN,Longitud,Ancho,Altura,Unidad,CC_Net_Change,Peso_perm_emb,Volumen_perm,Toleranc_sobrepeso,Toler_exceso_volum,UMP_variable,Sujeto_lote,Capac_llenado,Factor_apilado,Grupo_mat_ME,Valido_de1,Validez_a,Co_producto,StatMatTodosCe,Valido_de2,Valido_de3,Clasif_fiscal,Perfil_catalogo,TmpoHastaCaduc,DurTotalConserv,%TmpoCaducAlmac,Contenido_neto,Ctd_base_comp,Contenido_bruto,Metodo_convers,Utiliz_UM,Viscos_elevada,A_granel_liqu,Log_lote_nec,Niv_fabr_mater,Ind_periodo,Gr_tp_pos_gral,Fijado,Cl_lista_surt,Fe_cad_Fe_exp,Capacidad_max,Tol_sobrecap,Long_max,Ancho_per_emb,Alt_max,Per_cuarent,ANP_Code,Gr_unidad_carga,Categ_estruct,Tp_tol,Grupo_de_cuenta,Agrupacion_DSD,Unidad_de_medida_Volumen_Original,Numerador_para_conversion_Volumen_Original,Denominador_para_conversion_Volumen_Original,Unidad_de_medida_Volumen_Destino,Numerador_para_conversion_Volumen_Destino,Denominador_para_conversion_Volumen_Destino,Volumen,Jquia_productos_nivel_1,Jquia_productos_nivel_2,Jquia_productos_nivel_3
000000000010010108,400,0150190076,10,0,5600.0,ST,ST,5600.000,null,1.00000,1904.0,null,0,0073045652,10,0622061895,10,CHATARRA METALICA,null,null,5000,ENSERARMCJOTRAMAQ,null,MS,3207,CL,09,null,01,null,null,null,900,9000,00,null,null,0.00,1CL27331,null,0,null,null,null,null,null,PROYECTO01,2022.04.28,FCANADAS,KVCEXBDLZ,KVCEBDL,ZA60,A,null,null,0,null,null,null,CL01,1.0,1.0,KG,null,null,null,null,0.000,null,null,0.000,0.000,0.000,null,null,0.000,0.000,0.0,0.0,null,null,0,0,null,1900.01.01,1900.01.01,null,null,1900.01.01,1900.01.01,null,null,0,0,0,0.000,0,0.000,null,null,null,null,null,0,null,LEIS,null,null,B,0.000,0.0,0.000,0.000,0.000,0,0,null,null,null,null,null,ST,1.0,1.0,M3,1.0,1.0,5600.0,ENSER,ARMCJ,OTRAMAQ
M100PLY000122,400,0150113050,60,0,0.0,ST,ST,1944.000,1103,1.00000,0.0,null,0,0072991499,20,0622021433,60,PLY PIN BC DECO EXT 2440X12...,null,null,5000,YMPCLGRDOBRAN_S2S,CM53,M3,3207,CL,09,35,01,null,null,null,320,3201,00,null,001,0.00,1CL27331,YP4,60,YP1,YP2,null,null,null,CARGASINI,2022.05.09,MACUNAF,KELVBGDAPXCZ,KELVBGDAPC,ZFE1,A,null,null,0,9230009,null,260,CL06,13.53,13.53,KG,M1,0,0001,null,0.000,null,null,0.000,0.000,0.000,null,null,0.000,0.000,0.0,0.0,1,X,0,0,null,1900.01.01,1900.01.01,X,null,1900.01.01,1900.01.01,null,null,0,0,0,0.000,0,0.000,null,null,null,null,null,0,null,NORM,null,null,B,0.000,0.0,0.000,0.000,0.000,0,0,null,null,null,null,null,ST,1.0,1.0,M3,6756.0,181.0,0.0,YMPCL,GRDOB,RAN_S2S
M100PLY000129,400,0150113050,10,0,0.0,ST,ST,160.000,1103,1.00000,0.0,null,0,0072991499,10,0622021433,10,PLY PIN BC DECO EXT 2440X12...,null,null,5000,YMPCLGRDOBRAN_S2S,CM53,M3,3207,CL,09,35,01,null,null,null,320,3201,00,null,001,0.00,1CL27331,YP4,10,YP1,YP3,null,null,null,CARGASINI,2022.05.09,MACUNAF,KELVBGDAPXCZ,KELVBGDAPC,ZFE1,A,null,null,0,9230012,null,260,CL06,18.039,18.039,KG,M1,0,0001,null,0.000,null,null,0.000,0.000,0.000,null,null,0.000,0.000,0.0,0.0,1,X,0,0,null,1900.01.01,1900.01.01,X,null,1900.01.01,1900.01.01,null,null,0,0,0,0.000,0,0.000,null,null,null,null,null,0,null,NORM,null,null,B,0.000,0.0,0.000,0.000,0.000,0,0,null,null,null,null,null,ST,1.0,1.0,M3,92773.0,3314.0,0.0,YMPCL,GRDOB,RAN

In [11]:
df_materiales = dataframes_dict['materiales'].where(col('Mandante') == '400')

In [12]:
df_paises = dataframes_dict['paises'].where(col('Mandante') == '400').where(col('Idioma') == 'S')
# Ocupar sólo "S" Y info mandante 400

In [13]:
df_paises = dataframes_dict['paises'].where(col('Mandante') == '400')

## DF FILTRADOS POR MANDANTE = 400

In [14]:
df_metadata_facturas

Mandante,Sociedad,N_documento,Ejercicio,Clase_documento,Fecha_documento,Fecha_contab,Per_contable,Registrado_el,Modificado_el,Nombre_usuario,Referencia,Moneda,Moneda_grupo,Status_doc,Op_referencia,Clv_referencia,Entidad_CP,Moneda_local,Tipo_cotizacion1,Tipo_cotizacion2,Soc_desencaden,T_C_para_imptos,Motiv_anulacion,Num_de_tienda,Ctd_paginas,Ledger,Grupo_ledgers,FechDeclImpuest,Contab_dividida,Tipo_cotizacion,Tipo_de_orden,Motivo,Codigo_transaccion,Factura,Clase_factura,Organiz_ventas,Canal_distrib,Esquema_calculo,Condicion_doc,Condicion_expedicion,Fecha_factura,Grupo_precios,Grupo_clientes,Zona_de_ventas,Lista_precios,Incoterms,Incoterms_2,Status_contab,Camb_p_contab,Dias_val_adic,Fecha_de_valor,Cond_pago,Via_pago,Gpo_imputacion,Pais_receptor,Region,Cl_Fiscal_1_Clt,Valor_neto,Crit_agrupam,Creado_por,Hora,Creado_el,Pagador,Solicitante,Moneda_estad,N_com_exterior,DocFacturAnulad,Acuerdo,Cl_lista_fact,Fec_fact_lst_fc,Sector,Area_ctrl_cdto,Cta_credito,Cambio,JerTip_DetPrec,N_de_pedido,PSdeterm_impto,Origen_N_I_F,Pais_N_I_F,Asignacion,Impte_impuesto,Sistema_logico,Fe_conversion,TpCambCartCred,Refer_pago
400,127,0005047168,2020,KB,null,null,12,2020.12.01,2020.12.07,OPERACIONES,41279 1,CLP,null,null,CTE,41279 1 1,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047181,2020,HR,null,null,12,2020.12.01,1900.01.01,KTAPIA,FINI,CLP,null,null,BKPF,0005047181127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,FB01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047182,2020,HR,null,null,12,2020.12.01,1900.01.01,KTAPIA,FINI,CLP,null,null,BKPF,0005047182127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,FB01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047202,2020,HR,null,null,12,2020.12.01,1900.01.01,KTAPIA,FINI,CLP,null,null,BKPF,0005047202127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,FB01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
400,127,0005047335,2020,KA,null,null,12,2020.12.01,1900.01.01,AREBOLLEDO,3970823468-4,CLP,null,null,BKPFF,0005047335127 2020,null,CLP,M,M,null,0.00000,null,null,0,null,null,1900.01.01,null,M,null,null,ZFI1165,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [15]:
df_facturas_por_cliente

Numero_de_Cliente,Mandante,Factura,Posicion_de_Factura,Tipo_de_Cliente,Direccion,Punto_descarga,Clave_pais,Indicador_direccion,Descr_cliente_socio,Zona_transporte,Numero_personal,Cliente_Solicitante,Cliente_Pagador,Cliente_Destinatario_de_Mercancia,Cliente_Destinatario_de_factura,Representante_de_ventas,Pais,Nombre,Poblacion,Codigo_postal,Region,Conc_busq,Calle,Telefono,Nombre_1,Poblacion_1,Tratamiento,Creado_el,Creado_por,Puestos_de_descarga,Grupo_de_ceuntas,Acreedor,Nombre_3,Nombre_4,Distrito,Clave_de_idioma,N_ident_fis_1,Telefono_2,Zona_de_transporte,Sociedad_GL_asociada,Atributo_1,Persona_fisica
164952,400,0150190076,0,AG,0001299162,null,CL,D,null,08102,0,164952,null,null,null,null,CL,Reciclaje de Metales Ingeni...,CORONEL,null,08,REMET,Av Forestal # 947 Parque I...,41-2751070,RECICLAJE DE METALES INGE,CORONEL,Sres.,2016.12.28,MDIAZB,null,CL01,null,null,null,null,S,76368838-0,null,08102,null,null,null
164952,400,0150190076,0,RG,0001299162,null,CL,D,null,08102,0,null,164952,null,null,null,CL,Reciclaje de Metales Ingeni...,CORONEL,null,08,REMET,Av Forestal # 947 Parque I...,41-2751070,RECICLAJE DE METALES INGE,CORONEL,Sres.,2016.12.28,MDIAZB,null,CL01,null,null,null,null,S,76368838-0,null,08102,null,null,null
164952,400,0150190076,0,RE,0001299162,null,CL,D,null,08102,0,null,null,null,164952,null,CL,Reciclaje de Metales Ingeni...,CORONEL,null,08,REMET,Av Forestal # 947 Parque I...,41-2751070,RECICLAJE DE METALES INGE,CORONEL,Sres.,2016.12.28,MDIAZB,null,CL01,null,null,null,null,S,76368838-0,null,08102,null,null,null
null,400,0150190076,0,VE,null,null,CL,D,null,null,99999999,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,400,0150190076,10,VD,null,null,CL,D,null,null,99999999,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [16]:
df_materiales

Material,Mandante,Factura,Posicion,Posicion_sup,Ctd_facturada,UM_venta,UM_base,Ctd_necesaria,Division,Tp_cambio,Valor_neto,Causante,Posicion1,Doc_modelo,Posicion_modelo,Documento_venta,Posicion2,Denominacion,Material_precio,Lote,Grupo_articulos,Jquia_productos,Puesto_exped,Sector,Centro,Pais,Reg_ce_sum,Gr_materiales,Gr_imput_mat,Centro_coste,Grupo_rappels,Grupo_comision,Gr_vendedores,Oficina_ventas,Sector2,Cl_valoracion,Almacen,Costes_internos,CeBe,Gr_materiales_1,Asociacion_posicion_volumen,Gr_materiales_2,Gr_materiales_3,Plazo_entrega,Fecha_entrega,Creado_el,Creado_por,Ultima_modif,Modificado_por,Stat_act_comp,Stat_actual,Tipo_material,Ramo,N_Material_ant,UM_de_pedido,Numero_hojas,Fabr_insp,Tamano_Dimens,Denom_estandar,CvValoresCompr,Peso_bruto,Peso_neto,Unidad_de_peso,Cond_almacenaje,Nivel_plf_nec,Gpo_transporte,N_sust_peligr,Cant_vales_EM,Codigo_EAN_UPC,Tipo_EAN,Longitud,Ancho,Altura,Unidad,CC_Net_Change,Peso_perm_emb,Volumen_perm,Toleranc_sobrepeso,Toler_exceso_volum,UMP_variable,Sujeto_lote,Capac_llenado,Factor_apilado,Grupo_mat_ME,Valido_de1,Validez_a,Co_producto,StatMatTodosCe,Valido_de2,Valido_de3,Clasif_fiscal,Perfil_catalogo,TmpoHastaCaduc,DurTotalConserv,%TmpoCaducAlmac,Contenido_neto,Ctd_base_comp,Contenido_bruto,Metodo_convers,Utiliz_UM,Viscos_elevada,A_granel_liqu,Log_lote_nec,Niv_fabr_mater,Ind_periodo,Gr_tp_pos_gral,Fijado,Cl_lista_surt,Fe_cad_Fe_exp,Capacidad_max,Tol_sobrecap,Long_max,Ancho_per_emb,Alt_max,Per_cuarent,ANP_Code,Gr_unidad_carga,Categ_estruct,Tp_tol,Grupo_de_cuenta,Agrupacion_DSD,Unidad_de_medida_Volumen_Original,Numerador_para_conversion_Volumen_Original,Denominador_para_conversion_Volumen_Original,Unidad_de_medida_Volumen_Destino,Numerador_para_conversion_Volumen_Destino,Denominador_para_conversion_Volumen_Destino,Volumen,Jquia_productos_nivel_1,Jquia_productos_nivel_2,Jquia_productos_nivel_3
000000000010010108,400,0150190076,10,0,5600.0,ST,ST,5600.000,null,1.00000,1904.0,null,0,0073045652,10,0622061895,10,CHATARRA METALICA,null,null,5000,ENSERARMCJOTRAMAQ,null,MS,3207,CL,09,null,01,null,null,null,900,9000,00,null,null,0.00,1CL27331,null,0,null,null,null,null,null,PROYECTO01,2022.04.28,FCANADAS,KVCEXBDLZ,KVCEBDL,ZA60,A,null,null,0,null,null,null,CL01,1.0,1.0,KG,null,null,null,null,0.000,null,null,0.000,0.000,0.000,null,null,0.000,0.000,0.0,0.0,null,null,0,0,null,1900.01.01,1900.01.01,null,null,1900.01.01,1900.01.01,null,null,0,0,0,0.000,0,0.000,null,null,null,null,null,0,null,LEIS,null,null,B,0.000,0.0,0.000,0.000,0.000,0,0,null,null,null,null,null,ST,1.0,1.0,M3,1.0,1.0,5600.0,ENSER,ARMCJ,OTRAMAQ
M100PLY000122,400,0150113050,60,0,0.0,ST,ST,1944.000,1103,1.00000,0.0,null,0,0072991499,20,0622021433,60,PLY PIN BC DECO EXT 2440X12...,null,null,5000,YMPCLGRDOBRAN_S2S,CM53,M3,3207,CL,09,35,01,null,null,null,320,3201,00,null,001,0.00,1CL27331,YP4,60,YP1,YP2,null,null,null,CARGASINI,2022.05.09,MACUNAF,KELVBGDAPXCZ,KELVBGDAPC,ZFE1,A,null,null,0,9230009,null,260,CL06,13.53,13.53,KG,M1,0,0001,null,0.000,null,null,0.000,0.000,0.000,null,null,0.000,0.000,0.0,0.0,1,X,0,0,null,1900.01.01,1900.01.01,X,null,1900.01.01,1900.01.01,null,null,0,0,0,0.000,0,0.000,null,null,null,null,null,0,null,NORM,null,null,B,0.000,0.0,0.000,0.000,0.000,0,0,null,null,null,null,null,ST,1.0,1.0,M3,6756.0,181.0,0.0,YMPCL,GRDOB,RAN_S2S
M100PLY000129,400,0150113050,10,0,0.0,ST,ST,160.000,1103,1.00000,0.0,null,0,0072991499,10,0622021433,10,PLY PIN BC DECO EXT 2440X12...,null,null,5000,YMPCLGRDOBRAN_S2S,CM53,M3,3207,CL,09,35,01,null,null,null,320,3201,00,null,001,0.00,1CL27331,YP4,10,YP1,YP3,null,null,null,CARGASINI,2022.05.09,MACUNAF,KELVBGDAPXCZ,KELVBGDAPC,ZFE1,A,null,null,0,9230012,null,260,CL06,18.039,18.039,KG,M1,0,0001,null,0.000,null,null,0.000,0.000,0.000,null,null,0.000,0.000,0.0,0.0,1,X,0,0,null,1900.01.01,1900.01.01,X,null,1900.01.01,1900.01.01,null,null,0,0,0,0.000,0,0.000,null,null,null,null,null,0,null,NORM,null,null,B,0.000,0.0,0.000,0.000,0.000,0,0,null,null,null,null,null,ST,1.0,1.0,M3,92773.0,3314.0,0.0,YMPCL,GRDOB,RAN

In [25]:
df_paises

Mandante,Idioma,Pais,Pais_descr,Nacionalidad,Pais_descr_largo,Nacionalidad_largo,PRQ_SPREGT
400,D,RS,Russische Foed.,Russisch,null,null,null
400,E,RS,Russian Fed.,Russian,null,null,null
400,F,RS,Féd. russe,russe,null,null,null
400,S,RS,Serbia,serbia,null,null,null
400,S,SX,SINT MAARTEN,Sanmartinensa,null,null,null


## SELECT DE TABLAS

In [28]:
df_metadata_facturas.select('Mandante', 'Factura', 'Clase_factura',).where(col('Factura').isNotNull())


Mandante,Factura,Clase_factura
400,0150094014,YPBN
400,0150094258,YPBN
400,0150094579,YPBN
400,0150094602,YPBN
400,0150094686,YPBN


In [26]:
df_facturas_por_cliente.select('Mandante','Factura','Posicion_de_Factura','Pais')

Mandante,Factura,Posicion_de_Factura,Pais
400,0150190076,0,CL
400,0150190076,0,CL
400,0150190076,0,CL
400,0150190076,0,null
400,0150190076,10,null


In [29]:
df_materiales.select('Material', 'Volumen', 'Unidad_de_medida_Volumen_Original', 'Pais', 'Mandante', 'Factura')

Material,Volumen,Unidad_de_medida_Volumen_Original,Pais,Mandante,Factura
000000000010010108,5600.0,ST,CL,400,0150190076
M100PLY000122,0.0,ST,CL,400,0150113050
M100PLY000129,0.0,ST,CL,400,0150113050
M100PLY000122,2.8934280639431615,ST,CL,400,0150113050
M100PLY000122,2.8934280639431615,ST,CL,400,0150113050


In [30]:
df_paises.select('Mandante', 'Idioma', 'Pais' , 'Pais_descr')

Mandante,Idioma,Pais,Pais_descr
400,D,RS,Russische Foed.
400,E,RS,Russian Fed.
400,F,RS,Féd. russe
400,S,RS,Serbia
400,S,SX,SINT MAARTEN
